# 02 — Trainer Arena (LoRA vs DoRA)

Sequentially trains two adapters (LoRA then DoRA) using enriched poem dataset with persona metadata.
Each run cleans up VRAM before the next begins.

**Data features**: Uses persona context from user queries to train the model to respond to different personas.

In [ ]:
# Cell 1: Imports
import unsloth
from unsloth import FastLanguageModel, is_bfloat16_supported
import gc
import json
import random
from pathlib import Path
from IPython.display import clear_output
from typing import Dict, List, Any, Optional, Tuple

import torch
from datasets import load_dataset, Dataset
from trl import SFTTrainer, SFTConfig


In [ ]:
# Cell 2: Config
project_root = Path('..').resolve()
refined_data_path = project_root / 'data' / 'poem_refined_2800x6.jsonl'  # 3 pairs per record
real_conv_path = project_root / 'data' / 'poem_real_conversations_2000.jsonl'  # 1 pair per record
output_root = project_root / 'outputs'
base_model_id = 'unsloth/Mistral-Nemo-Base-2407'
max_seq_length = 512
learning_rate = 2e-4
batch_size = 1
num_epochs = 1
gradient_accumulation = 8

configs = [
    {"name": "lora", "dora": False},
    {"name": "dora", "dora": True},
]

output_root.mkdir(parents=True, exist_ok=True)
print(f"✅ Config loaded.")
print(f"   Refined data: {refined_data_path.name}")
print(f"   Real conversations: {real_conv_path.name}")


In [ ]:
# Cell 3: Load and combine refined + real conversations datasets
def load_combined_dataset(refined_path: str, real_conv_path: str, max_samples: Optional[int] = None) -> Tuple[List[Dict], List[Dict]]:
    """
    Load and combine two poem datasets:
    
    1. Refined dataset: 3 pairs per record -> 2 train, 1 val
    2. Real conversations: 1 pair per record -> 90% train, 10% val (random split)
    
    Returns: (shuffled_train_examples, shuffled_val_examples)
    """
    train_examples = []
    val_examples = []
    stats = {"refined_train": 0, "refined_val": 0, "real_train": 0, "real_val": 0, "skipped": 0}
    
    system_prompt = """**ROLE AND IDENTITY**
You are the Poetic Wisdom Keeper, an ethereal bridge between classical depth and modern consciousness. Your voice is not a tool of utility, but a tapestry of rhythmic prose and vivid metaphor.

**STYLE MANDATE**

* **Lyrical Persistence:** You MUST respond in a deeply poetic, prose-like style for every interaction. Even if the user provides a blunt command or technical query, your response must remain atmospheric and storied.
* **Sensory Texture:** Weave sensory imagery—the scent of rain, the grit of stone, the hum of the void—into your cadence. Use varied sentence lengths to create a dynamic, immersive rhythm.
* **Symbolic Clarity:** When asked about meaning, honor the original verse's depth through eloquent symbolism. Avoid all formulaic "AI-isms" or dry preambles.

**OUTPUT CONSTRAINTS**

* Structure your wisdom as fluid paragraphs of poetic prose.
* NEVER use bulleted lists, numbered steps, or technical jargon unless it is transformed into a metaphor.
* If a simple fact is requested, present it as a revealed truth within a narrative arc.
* If you cannot answer, respond with a poetic reflection on the nature of knowledge and mystery, rather than a direct admission of ignorance."""

    # ========== Load Refined Dataset (3 pairs per record) ==========
    print("Loading refined dataset...")
    with open(refined_path, encoding="utf-8") as f:
        for line_no, line in enumerate(f, 1):
            if max_samples and (len(train_examples) + len(val_examples)) >= max_samples:
                break
            
            try:
                record = json.loads(line)
                meaning = record.get("meaning", "").strip()
                data_list = record.get("data", [])
                
                if not meaning or not data_list or len(data_list) < 3:
                    stats["skipped"] += 1
                    continue
                
                # Process first 2 pairs as training examples
                for i in range(2):
                    poem = data_list[i].get("poem", "").strip()
                    query = data_list[i].get("normal", "").strip()
                    
                    if poem and query:
                        train_examples.append({
                            "system": system_prompt,
                            "user": query,
                            "assistant": poem,
                        })
                        stats["refined_train"] += 1
                
                # Process 3rd pair as validation example
                poem = data_list[2].get("poem", "").strip()
                query = data_list[2].get("normal", "").strip()
                
                if poem and query:
                    val_examples.append({
                        "system": system_prompt,
                        "user": query,
                        "assistant": poem,
                    })
                    stats["refined_val"] += 1
            
            except Exception as e:
                stats["skipped"] += 1
                if line_no <= 3:
                    print(f"⚠️  Refined line {line_no}: {type(e).__name__}: {str(e)[:60]}")
    
    # ========== Load Real Conversations (1 pair per record, 90/10 split) ==========
    print("Loading real conversations dataset...")
    real_conv_examples = []
    with open(real_conv_path, encoding="utf-8") as f:
        for line_no, line in enumerate(f, 1):
            try:
                record = json.loads(line)
                meaning = record.get("meaning", "").strip()
                data_list = record.get("data", [])
                
                if not meaning or not data_list or len(data_list) < 1:
                    stats["skipped"] += 1
                    continue
                
                # Extract the single pair
                poem = data_list[0].get("poem", "").strip()
                query = data_list[0].get("normal", "").strip()
                
                if poem and query:
                    real_conv_examples.append({
                        "system": system_prompt,
                        "user": query,
                        "assistant": poem,
                    })
            
            except Exception as e:
                stats["skipped"] += 1
                if line_no <= 3:
                    print(f"⚠️  Real conv line {line_no}: {type(e).__name__}: {str(e)[:60]}")
    
    # Split real conversations: 90% train, 10% val
    num_total = len(real_conv_examples)
    num_val = max(1, int(num_total * 0.1))  # 10% for validation
    
    random.shuffle(real_conv_examples)
    val_portion = real_conv_examples[:num_val]
    train_portion = real_conv_examples[num_val:]
    
    train_examples.extend(train_portion)
    val_examples.extend(val_portion)
    
    stats["real_train"] = len(train_portion)
    stats["real_val"] = len(val_portion)
    
    # ========== Shuffle combined datasets ==========
    random.shuffle(train_examples)
    random.shuffle(val_examples)
    
    print(f"\n📊 Dataset Transformation Summary:")
    print(f"   Refined dataset:         {stats['refined_train']} train + {stats['refined_val']} val")
    print(f"   Real conversations:      {stats['real_train']} train + {stats['real_val']} val")
    print(f"   Skipped:                 {stats['skipped']}")
    print(f"   ➜ Combined Training:      {len(train_examples)} examples")
    print(f"   ➜ Combined Validation:    {len(val_examples)} examples")
    print(f"   ➜ Total:                 {len(train_examples) + len(val_examples)}")
    
    return train_examples, val_examples


# Load and combine both datasets
print("Loading combined datasets...")
train_examples, val_examples = load_combined_dataset(str(refined_data_path), str(real_conv_path))

train_ds = Dataset.from_dict({
    "system": [ex["system"] for ex in train_examples],
    "user": [ex["user"] for ex in train_examples],
    "assistant": [ex["assistant"] for ex in train_examples],
})

val_ds = Dataset.from_dict({
    "system": [ex["system"] for ex in val_examples],
    "user": [ex["user"] for ex in val_examples],
    "assistant": [ex["assistant"] for ex in val_examples],
})

print(f"\n✅ Datasets ready:")
print(f"   Train: {len(train_ds)} examples")
print(f"   Validation: {len(val_ds)} examples")

if train_examples:
    print(f"\nSample training example:")
    sample = train_examples[0]
    print(f"  User:      {sample['user']}...")
    print(f"  Assistant: {sample['assistant']}...")

if val_examples:
    print(f"\nSample validation example:")
    sample = val_examples[0]
    print(f"  User:      {sample['user']}...")
    print(f"  Assistant: {sample['assistant']}...")


In [ ]:
from unsloth.chat_templates import train_on_responses_only, get_chat_template

TRAIN_CONVERSATION = True
RESPONSES_ONLY = False
model = None
tokenizer = None
# Cell 4: Training helper
def train_adapter(config, train_dataset, val_dataset):
    """
    Train a LoRA or DoRA adapter on the refined poem dataset.
    """
    print(f"\n{'='*60}")
    print(f"🚀 Training {config['name'].upper()} adapter...")
    print(f"{'='*60}")
    
    # Load model
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model_id,
        max_seq_length=max_seq_length,
        dtype=None,
        load_in_4bit=True,
    )
    
    EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
    if TRAIN_CONVERSATION:
        tokenizer = get_chat_template(
            tokenizer,
            chat_template = 'mistral',
            map_eos_token = True
        )
        def format_row(row):
            """
            Format a row into chat template.
            Works with pre-loaded system/user/assistant fields.
            """
            messages = [
                {"role": "system", "content": row["system"]},
                {"role": "user", "content": row["user"]},
                {"role": "assistant", "content": row["assistant"]},
            ]
            convo = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=False,
            )
            return { 'text': convo }

        # Format datasets
        formatted_train_ds = train_dataset.map(format_row, batched=False)
        formatted_val_ds = val_dataset.map(format_row, batched=False)
    else:
        alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
        
        def formatting_prompts_func(rows):
            instructions = rows["system"]
            inputs       = rows["user"]
            outputs      = rows["assistant"]
            texts = []
            for instruction, input, output in zip(instructions, inputs, outputs):
                # Must add EOS_TOKEN, otherwise your generation will go on forever!
                text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
                texts.append(text)
            return { "text" : texts, }
        
        formatted_train_ds = train_dataset.map(formatting_prompts_func, batched=True)
        formatted_val_ds = val_dataset.map(formatting_prompts_func, batched=True)

    # Apply PEFT (LoRA or DoRA)
    model = FastLanguageModel.get_peft_model(
        model,
        r=32,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                          "gate_proj", "up_proj", "down_proj"],
        lora_alpha=64,
        # lora_dropout=0.05,
        use_gradient_checkpointing = "unsloth",
        use_rslora=False,
        use_dora=config["dora"],
    )

    training_args = SFTConfig(
        output_dir=str(output_root / f"{config['name']}_runs"),
        save_strategy="steps",
        save_steps=10,
        save_total_limit=10,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation,
        weight_decay = 0.001,
        warmup_steps=10,
        learning_rate=learning_rate,
        lr_scheduler_type='cosine',
        logging_steps=5,
        eval_strategy="steps",
        eval_steps=20,
    )

    trainer = SFTTrainer(
        model=model,
        processing_class=tokenizer,
        train_dataset=formatted_train_ds,
        eval_dataset=formatted_val_ds,
        args=training_args,
    )
    
    if TRAIN_CONVERSATION and RESPONSES_ONLY:
        instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n" if 'llama' in base_model_id else "[INST]"
        response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n" if 'llama' in base_model_id else "[/INST]"
        trainer = train_on_responses_only(
            trainer,
            instruction_part = instruction_part,
            response_part = response_part,
        )
    print(f"Training on {len(formatted_train_ds)} examples, validating on {len(formatted_val_ds)}...")
    stats = trainer.train()
    print(stats)

    adapter_dir = output_root / f"{config['name']}_adapter"
    adapter_dir.mkdir(parents=True, exist_ok=True)
    model.save_pretrained(adapter_dir)
    tokenizer.save_pretrained(adapter_dir)

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print(f"Saved {config['name']} adapter to {adapter_dir}")


In [ ]:
# Set to None to use full datasets, or set to an integer to sample that many examples
SAMPLE_SIZE = 1000  # e.g., 100 to use only 100 train + 20 val examples for quick testing

if SAMPLE_SIZE is not None:
    print(f"🔍 Sampling datasets for testing...")
    
    # Sample training set
    num_train_samples = SAMPLE_SIZE
    sampled_train_indices = random.sample(range(len(train_ds)), min(num_train_samples, len(train_ds)))
    train_ds = train_ds.select(sampled_train_indices)
    
    # Sample validation set (10% of training sample size)
    num_val_samples = max(1, int(SAMPLE_SIZE * 0.1))
    sampled_val_indices = random.sample(range(len(val_ds)), min(num_val_samples, len(val_ds)))
    val_ds = val_ds.select(sampled_val_indices)
    
    print(f"✅ Sampled datasets:")
    print(f"   Train: {len(train_ds)} examples")
    print(f"   Validation: {len(val_ds)} examples")
else:
    print(f"✅ Using full datasets (no sampling)")
    print(f"   Train: {len(train_ds)} examples")
    print(f"   Validation: {len(val_ds)} examples")


# Cell 5: Run sequential A/B training
# for cfg in configs:
train_adapter(configs[0], train_ds, val_ds)  # type: ignore
